In [1]:
#Utilizacion de Word2Vec 

In [37]:
import bs4 as bs
import urllib.request
import re
import nltk
import pandas as pd
pd.set_option('max_colwidth', -1)
import string
import nltk
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('vader_lexicon')
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\brian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\brian\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [38]:
df = pd.read_csv('../dataset/train.csv')
df_test = pd.read_csv('../dataset/test.csv')
df.drop(columns=['location', 'keyword', 'target'], inplace=True)
df_test.drop(columns=['location', 'keyword'], inplace=True)
df = df.merge(df_test, how='outer')

In [39]:
#Al igual que en otros notebooks, preprocesamiento previo
# Quitamos las urls
df['text'] = df['text'].str.replace(r'http:\/\/.*', '', regex=True).replace(r'https:\/\/.*', '', regex=True)

# Quitamos user mentions, signos de puntuación, hashtags y stopwords.
def clean_text(text):
    words = text.lower().split(' ')
    text = ' '.join([word for word in words if not word.startswith('@') and word not in stopwords.words('english')])
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

df['text'] = df['text'].apply(clean_text)

df['word_count'] = df['text'].apply(lambda x: len(x.split(' ')))

In [40]:
df.head()

,id,text,word_count
0,1,deeds reason earthquake may allah forgive us,7
1,4,forest fire near la ronge sask canada,7
2,5,residents asked shelter place notified officers evacuation shelter place orders expected,11
3,6,13000 people receive wildfires evacuation orders california,8
4,7,got sent photo ruby alaska smoke wildfires pours school,10


In [41]:
#Para que esto funcione, se requiere que todas las palabras de todos los textos de los tweets esten en una lista
palabras = []
def extraer_palabras(string):
    splitted = string.split(' ')
    lista = []
    for palabra in splitted:
        lista.append(palabra)
    palabras.append(lista)
    
df['text'].apply(extraer_palabras)

from gensim.models import Word2Vec

word2vec = Word2Vec(palabras, min_count=2)
vocabulary = word2vec.wv.vocab
print(vocabulary)

{'deeds': <gensim.models.keyedvectors.Vocab object at 0x0000028E0B872198>, 'reason': <gensim.models.keyedvectors.Vocab object at 0x0000028E0CC18D68>, 'earthquake': <gensim.models.keyedvectors.Vocab object at 0x0000028E0CC18DA0>, 'may': <gensim.models.keyedvectors.Vocab object at 0x0000028E0CC18E10>, 'allah': <gensim.models.keyedvectors.Vocab object at 0x0000028E0CC18E48>, 'forgive': <gensim.models.keyedvectors.Vocab object at 0x0000028E0CC18E80>, 'us': <gensim.models.keyedvectors.Vocab object at 0x0000028E0CC18EB8>, 'forest': <gensim.models.keyedvectors.Vocab object at 0x0000028E0CC18EF0>, 'fire': <gensim.models.keyedvectors.Vocab object at 0x0000028E0CC18F28>, 'near': <gensim.models.keyedvectors.Vocab object at 0x0000028E0CC18F60>, 'la': <gensim.models.keyedvectors.Vocab object at 0x0000028E0CC18F98>, 'canada': <gensim.models.keyedvectors.Vocab object at 0x0000028E0CC18FD0>, 'residents': <gensim.models.keyedvectors.Vocab object at 0x0000028E0CC19048>, 'asked': <gensim.models.keyedvect

In [42]:
#Ahora, voy a aplicarle a cada una de las palabras, un average para crear la columna puntaje.

def calcular_promedio(text,string):
    split = string.split(' ')
    promedio = 0
    cantidad_de_palabras = 0
    for palabra in split:
        cantidad_de_palabras+=1
        try:
            promedio += word2vec.wv.similarity("debris",palabra)
        except KeyError:
            continue
    return promedio/cantidad_de_palabras    
    
df['promedio'] = df['text'].apply(lambda x: calcular_promedio(x,"debris"))
df

,id,text,word_count,promedio
0,1,deeds reason earthquake may allah forgive us,7,1.0
1,4,forest fire near la ronge sask canada,7,1.0
2,5,residents asked shelter place notified officers evacuation shelter place orders expected,11,1.0
3,6,13000 people receive wildfires evacuation orders california,8,1.0
4,7,got sent photo ruby alaska smoke wildfires pours school,10,1.0
5,8,rockyfire update california hwy 20 closed directions due lake county fire cafire wildfires,15,1.0
6,10,flood disaster heavy rain causes flash flooding streets manitou colorado springs areas,12,1.0
7,13,im top hill see fire woods,6,1.0
8,14,theres emergency evacuation happening building across street,7,1.0
9,15,im afraid tornado coming area,5,1.0
